# Workflow 4 - analyze the results
<hr>

1. Cluster interactive plots
2. Movie trajectory plots

In [ ]:
#required imports
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive, fixed, IntSlider, VBox, HBox, Layout
import ipywidgets as widgets
import matplotlib.patheffects as PathEffects
from IPython.display import display,clear_output
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from engens.core.EnGens import EnGen
from engens.core.ClustEn import *
from engens.core.FeatureSelector import *
from engens.core.DimReduction import *
from engens.core.PlotUtils import *
import pickle as pk
import os


In [ ]:
plt.ioff()

In [ ]:
engen = None
with open("wf3_resulting_EnGen.pickle", "rb") as file:
    engen = pk.load(file)
clust = None
with open("wf3_resulting_Clust.pickle", "rb") as file:
    clust = pk.load(file)

In [ ]:
%matplotlib inline

## Cluster plots
<hr>

In [ ]:
p_utils1 = PlotUtils(engen, clust, stride=20)
# if your trajectory is very large plotting this dashboard migth take some time
# because one point for every frame has to be plotted
# if you want a faster plot - provide the stride - run:
# p_utils1 = PlotUtils(engen, clust, stride=20)
# this will load every 20th frame from the trajectory
p_utils1.dashboard1(path = "./res_ensemble")

## View multiple structures per cluster
<hr>

In [ ]:
cluster_index = 0
n_frames = 5

In [ ]:
p_utils1.plot_frames_from_cluster(cluster_index, n_frames, 
                                  plot_representative=True)

In [ ]:
cluster_index = 2
n_frames = 5

In [ ]:
# You can ask for the sampled frames to be extracted into a folder of your choice
p_utils1.plot_frames_from_cluster(cluster_index, n_frames, 
                                  plot_representative=True,
                                 extract_selected=True,
                                folder_path="./cluster3_samples")

## Trajectory movie
<hr>

In [ ]:
p_utils1.view_movie("cartoon")

In [ ]:
p_utils1.view_movie("ball_and_stick")

## Additional info per cluster - distance between two residues
<hr>

In [ ]:
residue1_index = 0
residue2_index = 3

In [ ]:
from pyemma.coordinates import source

pyemma_src = source(engen.traj, top=engen.ref, chunksize=100)
pyemma_src.featurizer.add_residue_mindist([[residue1_index,residue2_index]])

data = p_utils1.load_as_memmap(pyemma_src)

In [ ]:
scatter_fig = p_utils1.plot_custom_feature_scatter(data[:, 0].flatten(), 
                                                   y_title="residue mindist {}-{}".format(residue1_index, residue2_index))

In [ ]:
box_fig = p_utils1.plot_custom_feature_box(data[:, 0].flatten(), 
                                                   y_title="residue mindist {}-{}".format(residue1_index, residue2_index))

## Additional info per cluster - RMSD to a frame
<hr>

In [ ]:
ref_frame = 3294

In [ ]:
from pyemma.coordinates import source

pyemma_src = source(engen.traj, top=engen.ref, chunksize=100)
pyemma_src.featurizer.add_minrmsd_to_ref(engen.traj, ref_frame=ref_frame)

data = p_utils1.load_as_memmap(pyemma_src)

In [ ]:
scatter_fig = p_utils1.plot_custom_feature_scatter(data[:, 0].flatten(), 
                                                   y_title="RMSD to frame {}".format(ref_frame))

In [ ]:
scatter_fig = p_utils1.plot_custom_feature_box(data[:, 0].flatten(), 
                                                   y_title="RMSD to frame {}".format(ref_frame))

## Additional info per cluster - RMSD to another structure (given a PDB file)
<hr>

In [ ]:
#ref_file = "./pentapeptide-00-500ns-impl-solv-aligned-engen-selected.pdb"

In [ ]:
from pyemma.coordinates import source

#pyemma_src = source(engen.traj, top=engen.ref, chunksize=100)
#pyemma_src.featurizer.add_minrmsd_to_ref(mdtraj.load(ref_file))

#data = p_utils1.load_as_memmap(pyemma_src)

In [ ]:
#scatter_fig = p_utils1.plot_custom_feature_scatter(data[:, 0].flatten(), 
                                                   y_title="RMSD to structure")

In [ ]:
#scatter_fig = p_utils1.plot_custom_feature_box(data[:, 0].flatten(), 
                                                   y_title="RMSD to frame {}".format(ref_frame))

## Additional info per cluster - component from dimensionality reduction
<hr>

In [ ]:
component_index = 2

In [ ]:
data = engen.dimred_data[:, component_index]

In [ ]:
scatter_fig = p_utils1.plot_custom_feature_scatter(data, 
                                                   y_title="Value of component {}".format(component_index))

In [ ]:
scatter_fig = p_utils1.plot_custom_feature_box(data, 
                                                   y_title="Value of component {}".format(component_index))